In [13]:
import dgl
import dgl.function as fn
import torch as torch
import torch.nn as nn
import torch.nn.functional as F
from dgl import DGLGraph
import tqdm as tqdm


# Graphs in DGL

In [93]:
edges = ([0, 0, 0, 0, 0, 1],
         [1, 2, 3, 4, 5, 5])
num_nodes = 6
num_edges = len(edges[0])

g = dgl.graph(edges, num_nodes=num_nodes)

g.edges()

(tensor([0, 0, 0, 0, 0, 1]), tensor([1, 2, 3, 4, 5, 5]))

In [94]:
g.ndata['x'] = torch.randn(num_nodes, 3)
g.edata['a'] = torch.randn(num_edges, 4)
# Assign a 5x4 node feature matrix for each node.  Node and edge features in DGL can be multi-dimensional.
g.ndata['y'] = torch.randn(num_nodes, 5, 4)

g.ndata['x']

tensor([[-0.7777, -0.0411,  0.0754],
        [ 0.3614,  0.4575, -0.2722],
        [ 1.7990, -0.3573, -1.6917],
        [-1.2815,  0.8051, -1.0646],
        [-0.9383,  1.3842, -0.4182],
        [-0.9762,  0.0391,  1.1020]])

## Extracting subgraphs

In [95]:
sg1 = g.subgraph([0,1,2])
sg1.num_nodes(), sg1.edges()

(3, (tensor([0, 0]), tensor([1, 2])))

In [96]:
sg2 = g.edge_subgraph([0,1,4])
sg2.num_nodes(), sg2.edges()

(4, (tensor([0, 0, 0]), tensor([1, 2, 3])))

## Load a big graph and features for it

In [97]:
from dgl.data import CoraGraphDataset
def load_cora_data():
    dataset = CoraGraphDataset()
    g = dataset[0]
    features = g.ndata['feat']
    labels = g.ndata['label']
    train_mask = g.ndata['train_mask']
    test_mask = g.ndata['test_mask']
    return g, features, labels, train_mask, test_mask
g_cora, features, labels, train_mask, test_mask = load_cora_data()

  NumNodes: 2708
  NumEdges: 10556
  NumFeats: 1433
  NumClasses: 7
  NumTrainingSamples: 140
  NumValidationSamples: 500
  NumTestSamples: 1000
Done loading data from cached files.


In [98]:
g_cora, features.shape

(Graph(num_nodes=2708, num_edges=10556,
       ndata_schemes={'feat': Scheme(shape=(1433,), dtype=torch.float32), 'label': Scheme(shape=(), dtype=torch.int64), 'val_mask': Scheme(shape=(), dtype=torch.bool), 'test_mask': Scheme(shape=(), dtype=torch.bool), 'train_mask': Scheme(shape=(), dtype=torch.bool)}
       edata_schemes={'__orig__': Scheme(shape=(), dtype=torch.int64)}),
 torch.Size([2708, 1433]))

# Operations on Graph

In [99]:
gcn_msg = fn.copy_u(u='h', out='m')
gcn_reduce = fn.sum(msg='m', out='out')

In [100]:
g.edges()

(tensor([0, 0, 0, 0, 0, 1]), tensor([1, 2, 3, 4, 5, 5]))

In [107]:
g.ndata['h'] = torch.Tensor([1, 0, 0, 0, 0, 1])
g.ndata['h'].shape

torch.Size([6])

In [109]:
g.ndata['h'].sum()

tensor(2.)

In [110]:
g.update_all(gcn_msg, gcn_reduce)
g.ndata['out'].sum()

tensor(5.)

In [114]:
print(g.ndata['h'], '\n', g.ndata['out'])

tensor([1., 0., 0., 0., 0., 1.]) 
 tensor([0., 1., 1., 1., 1., 1.])


### On big graph

In [74]:
g_cora.ndata['h'] = features
g_cora.ndata['h'].shape

torch.Size([2708, 1433])

In [75]:
g_cora.ndata['h'].sum()

tensor(2708.0002)

In [76]:
g_cora.update_all(gcn_msg, gcn_reduce)
g_cora.ndata['out'].sum()

tensor(10556.)

# GCN

In [5]:

gcn_msg = fn.copy_u(u='h', out='m')
gcn_reduce = fn.sum(msg='m', out='h')

class GCNLayer(nn.Module):
    def __init__(self, in_feats, out_feats):
        super(GCNLayer, self).__init__()
        self.linear = nn.Linear(in_feats, out_feats)

    def forward(self, g, feature):
        # Creating a local scope so that all the stored ndata and edata
        # (such as the `'h'` ndata below) are automatically popped out
        # when the scope exits.
        with g.local_scope():
            g.ndata['h'] = feature
            g.update_all(gcn_msg, gcn_reduce)
            h = g.ndata['h']
            return self.linear(h)